<a href="https://colab.research.google.com/github/shahzad-r1zv1/LocalLLM_experiments/blob/main/7S_vLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===============================================================
# 📦 ENVIRONMENT SETUP
# ===============================================================
!pip install requests tqdm

import os, sys, time, json, shlex, re, subprocess, requests, threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict


In [2]:
# ===============================================================
# 🚀 OLLAMA INSTALL + SERVER START
# ===============================================================

def run(cmd, check=True, quiet=False):
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if not quiet: print(p.stdout.strip())
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout.strip()

def ollama_installed():
    try:
        out = run("ollama --version", check=False, quiet=True)
        return "ollama" in out.lower()
    except Exception:
        return False

def ollama_running():
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
        return r.status_code == 200
    except Exception:
        return False

if not ollama_installed():
    print("📦 Installing Ollama (this may take a minute)...")
    !curl -fsSL https://ollama.com/install.sh | sh
    print("✅ Installation complete.")

if not ollama_running():
    print("🚀 Starting Ollama server...")
    logf = open("/tmp/ollama_serve.log", "w")
    subprocess.Popen(["ollama","serve"], stdout=logf, stderr=subprocess.STDOUT, text=True)
    for _ in range(60):
        if ollama_running(): break
        print(".", end="", flush=True); time.sleep(2)
    print("\n✅ Ollama API ready at http://127.0.0.1:11434")
else:
    print("✅ Ollama already running.")


📦 Installing Ollama (this may take a minute)...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
✅ Installation complete.
🚀 Starting Ollama server...
.
✅ Ollama API ready at http://127.0.0.1:11434


In [3]:
# ===============================================================
# 💬 LLM CHAT / GENERATE WRAPPER
# ===============================================================

BASE = "http://127.0.0.1:11434"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"
USE_GENERATE = True
TEMPERATURE, TOP_P, REPEAT_PENALTY, TIMEOUT_S = 0.5, 0.9, 1.1, 900

def ask_llm(model: str, sys: str, user: str) -> str:
    """Ask a local or remote LLM."""
    try:
        if USE_GENERATE:
            prompt = f"<<SYS>>\n{sys}\n<</SYS>>\n\n{user}"
            r = requests.post(GEN, json={
                "model": model, "prompt": prompt, "stream": False,
                "options": {"temperature": TEMPERATURE, "top_p": TOP_P, "repeat_penalty": REPEAT_PENALTY}
            }, timeout=TIMEOUT_S)
            r.raise_for_status()
            return r.json().get("response","").strip()
        else:
            r = requests.post(CHAT, json={
                "model": model,
                "messages":[{"role":"system","content":sys},{"role":"user","content":user}],
                "stream":False,
                "options":{"temperature":TEMPERATURE,"top_p":TOP_P,"repeat_penalty":REPEAT_PENALTY}
            }, timeout=TIMEOUT_S)
            r.raise_for_status()
            return r.json().get("message",{}).get("content","").strip()
    except Exception as e:
        print(f"❌ LLM error for {model}: {e}")
        return ""


In [4]:
# ===============================================================
# 🧠 MULTI-MODEL CONSULTANT + MANAGER ARCHITECTURE
# ===============================================================

CONSULTANT_MODELS = ["mistral", "gemma2:2b", "llama3"]
MANAGER_MODEL = "mistral"

MODEL_ROLES = {
    "mistral": "Analytical / strategy reasoning",
    "gemma2:2b": "Empathetic / cultural and people insights",
    "llama3": "Executive tone / synthesis & communication clarity"
}

def run_model_for_prompt(model, sys_prompt, user_prompt, prompt_name, out_dir="results"):
    os.makedirs(out_dir, exist_ok=True)
    fname = os.path.join(out_dir, f"{prompt_name.replace(' ','_')}_{model.replace(':','_')}.json")
    if os.path.exists(fname):
        print(f"⏩ Skipping {model} ({prompt_name}) — already exists.")
        return json.load(open(fname))

    print(f"🧩 {model:<10} | Running {prompt_name}")
    start = time.time()
    result = ask_llm(model, sys_prompt, user_prompt)
    dur = round(time.time()-start,1)
    data = {
        "prompt": prompt_name, "model": model,
        "role": MODEL_ROLES.get(model,"Consultant"),
        "response": result, "duration_s": dur
    }
    json.dump(data, open(fname,"w"), indent=2)
    print(f"✅ {model:<10} done in {dur}s → {fname}")
    return data

def synthesize_prompt_results(prompt_name, out_dir="results"):
    inputs = []
    for m in CONSULTANT_MODELS:
        f = os.path.join(out_dir, f"{prompt_name.replace(' ','_')}_{m.replace(':','_')}.json")
        if os.path.exists(f):
            inputs.append(json.load(open(f)))
    if not inputs:
        print(f"⚠️ No consultant outputs found for {prompt_name}")
        return

    consolidated = os.path.join(out_dir, f"{prompt_name.replace(' ','_')}_final.json")
    if os.path.exists(consolidated):
        print(f"⏩ Final already exists for {prompt_name}")
        return json.load(open(consolidated))

    print(f"\n🧠 Manager ({MANAGER_MODEL}) synthesizing {prompt_name}...")
    sys = "You are a senior WRC partner consolidating multiple consultant analyses."
    user = f"""Prompt: {prompt_name}

Combine the following consultant reports into a single executive-ready synthesis (do not indicate these are combinations of other reports):
{json.dumps(inputs, indent=2, ensure_ascii=False)}"""
    result = ask_llm(MANAGER_MODEL, sys, user)
    out = {"prompt": prompt_name, "model": MANAGER_MODEL, "response": result}
    json.dump(out, open(consolidated,"w"), indent=2)
    print(f"✅ Final synthesis complete → {consolidated}")
    return out


In [5]:
# ===============================================================
# 🧾 DEFINE PROMPTS (Example)
# ===============================================================

# Organization inputs used to fill prompt placeholders
ORG = {
    "name": "QA Automation Org",
    "industry": "Software / DevTools",
    "size_employees": 180,
    "annual_revenue": "N/A (cost center)",
    "locations": "Toronto; Remote",
    "current_context": "Transitioning from phased QA to DevSecOps; aiming for predictive quality and CI/CD-first culture.",
    "key_challenges": "Inconsistent test coverage; siloed defect data; unclear decision rights; skills gap in pipeline engineering.",
    "recent_changes": "Trunk-based dev in 2 squads; feature flags/canary pilots; Playwright + contract tests on 2 critical APIs.",
    # Optional fields for specific prompts (leave as "" if unknown)
    "strategy_statement": "Ship faster with confidence via automated quality gates and proactive risk analytics.",
    "strategic_goals": "12 months: 70% automated coverage; 24m: 95% critical path; MTTR<1hr; DORA elite.",
    "kpis": "DORA metrics, escaped defect rate, mean time to detect, e2e reliability SLOs.",
    "market_position": "Internal enablement platform within a large enterprise.",
    "competitive_advantages": "Domain expertise; test data virtualization; unified telemetry.",
    "target_segments": "Product teams building microservices/APIs.",
    "value_proposition": "Fewer incidents, faster releases, higher trust in automation.",
    "org_chart": "VP Eng → Dir QA/Platform → QA Enablement, SDET, Observability pods; squads map to product lines.",
    "layers": "VP → Director → Managers → ICs (4 layers)",
    "span_of_control": "Managers: 6-8 ICs avg",
    "decision_rights": "Product owns scope; QA owns quality standards; Dev owns implementation; shared release gates.",
    "xfunc": "Release Council; Reliability Guild; Architecture Forum.",
    "geo": "Toronto hub + distributed remote",
    "restructuring": "Evolving towards platform team model; decoupling test infra from app squads."
}

# Which prompts to run (choose from keys below)
RUN_PROMPTS = [
    "PROMPT_1_FULL_7S",
    "PROMPT_2_STRATEGY",
    "PROMPT_3_STRUCTURE",
    "PROMPT_4_SYSTEMS",
    "PROMPT_5_SHARED_VALUES",
    "PROMPT_6_SKILLS",
    "PROMPT_7_STYLE",
    "PROMPT_8_STAFF",
    "PROMPT_9_ALIGNMENT",
    "PROMPT_10_CHANGE",
    "PROMPT_11_DIGITAL",
    "PROMPT_12_BENCH",
    "PROMPT_13_GAPS",
    "PROMPT_14_INTEGRATION",
    "PROMPT_15_EXEC_SUMMARY",
    "MEGA_PROMPT"
]

prompts =  {
"PROMPT_1_FULL_7S": """Act as a WRC (World Renowned Consultancy) consultant conducting a comprehensive 7S analysis.
INPUTS NEEDED:
- Company/Organization name: [NAME]
- Industry: [INDUSTRY]
- Company size (employees): [NUMBER]
- Annual revenue: [REVENUE]
- Geographic presence: [LOCATIONS]
- Current business context: [BRIEF DESCRIPTION]
- Key challenges facing the organization: [LIST 3-5]
- Recent major changes or initiatives: [DESCRIBE]
Analyze all seven elements:
1. Strategy (competitive positioning, strategic priorities, value proposition)
2. Structure (org design, reporting lines, decision rights)
3. Systems (processes, IT systems, workflows)
4. Shared Values (culture, core beliefs, mission/vision)
5. Style (leadership approach, management practices)
6. Staff (workforce composition, capabilities, engagement)
7. Skills (organizational competencies, competitive advantages)
OUTPUT FORMAT:
- Executive summary with key findings
- Detailed analysis of each S (current state assessment)
- Interconnections and dependencies between elements
- Alignment score (1-10) for each element pair
- Top 5 misalignments requiring attention
- Recommendations prioritized by impact and feasibility
""",
"PROMPT_2_STRATEGY": """Evaluate the clarity and effectiveness of our organizational strategy using the 7S model.
INPUTS NEEDED:
- Current strategy statement: [PROVIDE]
- Strategic goals (3-5 year): [LIST]
- Key performance indicators: [LIST]
- Market position: [DESCRIBE]
- Competitive advantages: [LIST]
- Target customer segments: [DESCRIBE]
- Value proposition: [STATEMENT]
Assess our strategy by examining:
1. Clarity and communication throughout organization
2. Alignment with market opportunities
3. Differentiation from competitors
4. Resource allocation alignment
5. Measurability and tracking mechanisms
6. Connection to other 6 S elements
OUTPUT:
- Strategy effectiveness score (1-10) with justification
- Strengths and weaknesses analysis
- Gaps between stated and actual strategy
- Impact assessment on other 6 S elements
- 5 specific recommendations to strengthen strategy
- Communication plan to improve strategy understanding
""",
"PROMPT_3_STRUCTURE": """Analyze our organizational structure's alignment with strategic goals.
INPUTS NEEDED:
- Current org chart: [DESCRIBE HIERARCHY]
- Number of management layers: [NUMBER]
- Span of control averages: [NUMBERS]
- Decision-making authority levels: [DESCRIBE]
- Cross-functional teams/committees: [LIST]
- Geographic/divisional structure: [DESCRIBE]
- Recent restructuring efforts: [DESCRIBE IF ANY]
Evaluate:
1. Structure-strategy fit
2. Decision-making speed and effectiveness
3. Communication flow efficiency
4. Collaboration barriers
5. Duplication or gaps in responsibilities
6. Flexibility for future growth
OUTPUT:
- Structure effectiveness rating with evidence
- Organizational design recommendations
- Proposed org chart modifications
- Impact analysis on systems and staff
- Implementation roadmap for structural changes
- Risk assessment of proposed changes
""",
"PROMPT_4_SYSTEMS": """Assess the effectiveness of operational and management systems.
INPUTS NEEDED:
- Core business processes: [LIST]
- IT systems and platforms: [LIST]
- Performance management systems: [DESCRIBE]
- Financial/budgeting processes: [DESCRIBE]
- Quality control systems: [DESCRIBE]
- Communication systems: [DESCRIBE]
- Decision-making processes: [DESCRIBE]
- Knowledge management systems: [DESCRIBE]
Analyze:
1. Process efficiency and effectiveness
2. System integration and data flow
3. Automation opportunities
4. Performance tracking capabilities
5. User satisfaction and adoption
6. Alignment with strategic objectives
OUTPUT:
- Systems maturity assessment (1-5 scale per system)
- Critical system gaps and redundancies
- Process optimization opportunities
- Technology upgrade recommendations
- Implementation priority matrix
- ROI estimates for system improvements
""",
"PROMPT_5_SHARED_VALUES": """Identify and evaluate core shared values driving culture and decision-making.
INPUTS NEEDED:
- Stated mission/vision/values: [PROVIDE]
- Employee survey results: [KEY FINDINGS]
- Leadership behaviors observed: [EXAMPLES]
- Decision-making patterns: [DESCRIBE]
- Customer feedback themes: [SUMMARIZE]
- Internal communication samples: [PROVIDE EXAMPLES]
- Recognition and reward criteria: [LIST]
Examine:
1. Stated vs. lived values gap analysis
2. Values alignment across hierarchy
3. Values impact on behaviors
4. Cultural strengths and toxicities
5. Values-strategy alignment
6. Employee value perception
OUTPUT:
- Core values identification (top 5 actual vs. stated)
- Cultural health score with supporting evidence
- Values-behavior alignment matrix
- Cultural transformation requirements
- Values reinforcement action plan
- Measurement framework for cultural change
""",
"PROMPT_6_SKILLS": """Evaluate critical skills and competencies across the organization.
INPUTS NEEDED:
- Current workforce skills inventory: [CATEGORIES]
- Strategic capability requirements: [LIST]
- Competitor capabilities: [BENCHMARK DATA]
- Training and development programs: [DESCRIBE]
- Performance review data: [KEY METRICS]
- Skills gaps identified by managers: [LIST]
- Future skill requirements (3-5 years): [ANTICIPATE]
Assess:
1. Current vs. required skills gaps
2. Core competency strengths
3. Competitive skill advantages/disadvantages
4. Skills development effectiveness
5. Knowledge transfer mechanisms
6. Innovation and adaptation capabilities
OUTPUT:
- Skills heat map (current vs. required)
- Critical skills gap analysis with risk levels
- Competency development roadmap
- Make/buy/partner talent decisions
- L&D investment recommendations
- Skills KPIs and tracking mechanisms
""",
"PROMPT_7_STYLE": """Analyze the dominant leadership style and its impact on performance.
INPUTS NEEDED:
- Leadership team composition: [DESCRIBE]
- Leadership assessment data: [IF AVAILABLE]
- Employee engagement scores: [PROVIDE]
- Decision-making examples: [3-5 CASES]
- Communication patterns: [DESCRIBE]
- Change management approaches: [EXAMPLES]
- Succession planning status: [DESCRIBE]
Evaluate:
1. Predominant leadership styles
2. Leadership effectiveness metrics
3. Style-strategy alignment
4. Leadership impact on culture
5. Decision-making patterns
6. Leadership development needs
OUTPUT:
- Leadership style profile with strengths/weaknesses
- Leadership effectiveness score (1-10)
- Style-situation fit analysis
- Leadership development priorities
- Succession planning recommendations
- Leadership behavior change roadmap
""",
"PROMPT_8_STAFF": """Review workforce composition, recruitment, and retention strategies.
INPUTS NEEDED:
- Total headcount and demographics: [PROVIDE]
- Organizational structure by function: [BREAKDOWN]
- Turnover rates by level/function: [DATA]
- Time-to-fill metrics: [AVERAGES]
- Employee engagement scores: [PROVIDE]
- Compensation benchmarking: [POSITION VS MARKET]
- Talent pipeline status: [DESCRIBE]
- Diversity metrics: [PROVIDE]
Analyze:
1. Workforce composition vs. strategic needs
2. Talent acquisition effectiveness
3. Retention risks and drivers
4. Engagement and productivity levels
5. Diversity, equity, and inclusion status
6. Workforce planning adequacy
OUTPUT:
- Workforce health scorecard
- Critical talent risks and mitigation plans
- Recruitment strategy optimization
- Retention program enhancements
- Workforce planning recommendations
- HR metrics dashboard design
""",
"PROMPT_9_ALIGNMENT": """Evaluate how well all seven elements of the 7S Framework align.
INPUTS NEEDED:
- Brief assessment of each S element: [PROVIDE STATUS]
- Recent organizational changes: [LIST]
- Performance metrics trends: [LAST 2 YEARS]
- Strategic priorities: [TOP 5]
- Known pain points: [DESCRIBE]
- Success stories: [EXAMPLES]
Assess:
1. Element interdependencies and conflicts
2. Alignment scoring for each element pair (21 combinations)
3. Reinforcing vs. conflicting relationships
4. Impact of misalignments on performance
5. Root cause analysis of gaps
6. Synergy opportunities
OUTPUT:
- 7S alignment matrix with scores
- Critical misalignment identification
- Dependency map visualization
- Prioritized realignment initiatives
- Change sequencing recommendations
- Alignment monitoring framework
""",
"PROMPT_10_CHANGE": """Use the 7S Framework to analyze organizational readiness for change.
INPUTS NEEDED:
- Planned change initiative: [DESCRIBE]
- Change timeline and scope: [PROVIDE]
- Previous change efforts: [SUCCESS/FAILURE EXAMPLES]
- Stakeholder groups affected: [LIST]
- Current change capability maturity: [ASSESS 1-5]
- Resource availability: [BUDGET/PEOPLE]
- Risk tolerance: [LOW/MEDIUM/HIGH]
Analyze each S element for:
1. Current state change readiness
2. Required changes per element
3. Resistance points and drivers
4. Change capability gaps
5. Success enablers and barriers
6. Change impact assessment
OUTPUT:
- Change readiness score by S element
- Resistance heat map
- Change impact assessment matrix
- Stakeholder engagement strategy
- Change roadmap with quick wins
- Risk mitigation plan
- Success metrics framework
""",
"PROMPT_11_DIGITAL": """Apply the 7S Framework to assess digital transformation readiness and impact.
INPUTS NEEDED:
- Current digital maturity: [ASSESS 1-5]
- Digital strategy/initiatives: [DESCRIBE]
- Technology infrastructure: [CURRENT STATE]
- Digital skills inventory: [ASSESS]
- Data and analytics capabilities: [DESCRIBE]
- Customer digital expectations: [SUMMARIZE]
- Competitor digital positioning: [BENCHMARK]
Evaluate digital impact on:
1. Strategy (digital business models)
2. Structure (agile organization needs)
3. Systems (technology architecture)
4. Shared Values (digital culture)
5. Style (digital leadership)
6. Staff (digital talent)
7. Skills (digital capabilities)
OUTPUT:
- Digital maturity assessment by S element
- Digital transformation gaps and priorities
- Technology investment recommendations
- Digital culture transformation plan
- Reskilling/upskilling requirements
- Digital KPIs and governance model
- Transformation roadmap with milestones
""",
"PROMPT_12_BENCH": """Compare our 7S profile against key competitors.
INPUTS NEEDED:
- Top 3-5 competitors: [LIST]
- Competitive intelligence available: [SUMMARIZE]
- Industry best practices: [DESCRIBE]
- Our performance vs. competitors: [METRICS]
- Competitive advantages/disadvantages: [LIST]
- Market position: [DESCRIBE]
Benchmark:
1. Strategy effectiveness comparison
2. Organizational agility assessment
3. Operational excellence comparison
4. Cultural differentiation analysis
5. Leadership capability comparison
6. Talent competitiveness evaluation
7. Innovation capability assessment
OUTPUT:
- Competitive 7S comparison matrix
- Competitive advantage/disadvantage analysis
- Best practice identification
- Competitive gaps requiring closure
- Differentiation opportunities
- Competitive response strategies
- Monitoring and intelligence framework
""",
"PROMPT_13_GAPS": """Identify gaps between current and desired future state across the 7S elements.
INPUTS NEEDED:
- Vision for future state (3-5 years): [DESCRIBE]
- Current state assessment: [SUMMARIZE BY S]
- Strategic objectives: [LIST]
- Performance targets: [SPECIFY]
- Market/industry trends: [IDENTIFY]
- Stakeholder expectations: [DESCRIBE]
Analyze:
1. Current state baseline for each S
2. Future state requirements per S
3. Gap magnitude and complexity
4. Interdependency impact analysis
5. Resource requirements for gap closure
6. Timeline and sequencing needs
OUTPUT:
- Current vs. future state comparison table
- Gap severity assessment (critical/high/medium/low)
- Gap closure difficulty matrix
- Investment requirements estimate
- Transformation roadmap with phases
- Quick wins vs. long-term initiatives
- Success metrics and milestones
""",
"PROMPT_14_INTEGRATION": """Combine 7S insights with SWOT or PESTLE analysis.
INPUTS NEEDED:
- 7S assessment summary: [PROVIDE]
- SWOT analysis: [IF AVAILABLE]
- PESTLE factors: [IF AVAILABLE]
- Strategic options under consideration: [LIST]
- Risk factors identified: [LIST]
- Opportunity areas: [DESCRIBE]
Integrate analyses to:
1. Map external factors to internal capabilities
2. Identify strategic option feasibility
3. Assess implementation capabilities
4. Determine competitive positioning
5. Evaluate risk mitigation capacity
6. Prioritize strategic initiatives
OUTPUT:
- Integrated strategy framework
- Strategic option evaluation matrix
- Capability-opportunity alignment map
- Risk-readiness assessment
- Strategic initiative prioritization
- Implementation feasibility scores
- Integrated dashboard design
""",
"PROMPT_15_EXEC_SUMMARY": """Create an executive-level summary of 7S analysis with actionable recommendations.
INPUTS NEEDED:
- Full 7S analysis results: [PROVIDE KEY FINDINGS]
- Strategic context and urgency: [DESCRIBE]
- Available resources: [BUDGET/CAPACITY]
- Board/Executive priorities: [LIST]
- Key stakeholder concerns: [IDENTIFY]
- Success criteria: [DEFINE]
Synthesize:
1. Critical insights from 7S analysis
2. Top 3-5 strategic imperatives
3. Quick wins vs. transformational changes
4. Investment requirements and ROI
5. Risk assessment and mitigation
6. Implementation timeline
OUTPUT:
- 2-page executive summary
- Visual 7S alignment dashboard
- Top 10 recommendations ranked by impact
- Investment and resource requirements
- 90-day, 6-month, and 1-year action plans
- Success metrics and governance model
- Key risks and mitigation strategies
- Next steps and decision requirements
""",
"MEGA_PROMPT": """Act as a senior WRC consultant conducting a comprehensive 7S analysis. Provide a complete organizational assessment with actionable insights.
[Use all context provided above and infer reasonable assumptions where N/A.]
DELIVERABLES:
- Executive Summary; Detailed Analysis by S; Integrated Findings; Strategic Recs; Implementation Plan; Appendices.
"""
}

# Map bracketed placeholders to ORG fields
PLACEHOLDER_MAP = {
    "[NAME]": ORG.get("name",""),
    "[INDUSTRY]": ORG.get("industry",""),
    "[NUMBER]": ORG.get("size_employees",""),
    "[REVENUE]": ORG.get("annual_revenue",""),
    "[LOCATIONS]": ORG.get("locations",""),
    "[BRIEF DESCRIPTION]": ORG.get("current_context",""),
    "[LIST 3-5]": ORG.get("key_challenges",""),
    "[DESCRIBE]": ORG.get("recent_changes",""),
    "[PROVIDE]": ORG.get("strategy_statement",""),
    "[LIST]": ORG.get("strategic_goals",""),
    "[STATEMENT]": ORG.get("value_proposition",""),
    "[DESCRIBE HIERARCHY]": ORG.get("org_chart",""),
    "[NUMBERS]": ORG.get("span_of_control",""),
    "[IF AVAILABLE]": "",
    "[AVERAGES]": "",
    "[BREAKDOWN]": "",
    "[DATA]": "",
    "[POSITION VS MARKET]": "",
    "[ASSESS 1-5]": "",
    "[CURRENT STATE]": "",
    "[ASSESS]": "",
    "[SUMMARIZE]": "",
    "[ANALYZE]": "",
    "[IDENTIFY]": "",
    "[SPECIFY]": ""
}

# ===============================================================
# 🧾 FINALIZED PROMPT CONVERSION (dict ➜ tuple list)
# ===============================================================

def fill_placeholders(text: str, mapping: dict) -> str:
    """Replace bracketed placeholders (e.g. [NAME]) with ORG values."""
    for key, val in mapping.items():
        text = text.replace(key, str(val))
    return text

# Optional: a single reusable system prompt for all consultants
DEFAULT_SYS_PROMPT = (
    "You are a senior WRC (World Renowned Consultancy) consultant performing an advanced organizational analysis. "
    "Be precise, structured, and evidence-based. Use business reasoning, not generalities."
)

# Build list of tuples (name, system, user)
prompts_tuples = []
for key in RUN_PROMPTS:
    if key not in prompts:
        print(f"⚠️ Skipping missing prompt: {key}")
        continue

    user_text = fill_placeholders(prompts[key], PLACEHOLDER_MAP)
    prompts_tuples.append((key, DEFAULT_SYS_PROMPT, user_text))

# Now assign this as the final list used by the execution loop
prompts = prompts_tuples

print(f"✅ Prepared {len(prompts)} prompts ready for multi-model execution.")
for i, (name, sys_p, usr_p) in enumerate(prompts[:3]):
    print(f"{i+1}. {name}")



✅ Prepared 16 prompts ready for multi-model execution.
1. PROMPT_1_FULL_7S
2. PROMPT_2_STRATEGY
3. PROMPT_3_STRUCTURE


In [10]:
# ===============================================================
# 📥 PULL REQUIRED MODELS
# ===============================================================
import subprocess, time

MODELS = ["mistral", "gemma2:2b", "llama3"]
for model in MODELS:
    print(f"\n⬇️ Pulling {model}...")
    proc = subprocess.Popen(["ollama", "pull", model], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in proc.stdout:
        if "%" in line or "pulling" in line.lower():
            print(line.strip())
    proc.wait()
    if proc.returncode == 0:
        print(f"✅ {model} ready!")
    else:
        print(f"❌ Failed to pull {model}")


Streaming output truncated to the last 5000 lines.
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.6 MB/s    5m1spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.6 MB/s    5m1spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.6 MB/s    5m1spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.6 MB/s    5m1spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.8 MB/s   4m43spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.8 MB/s   4m43spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.8 MB/s   4m43spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.8 MB/s   4m43spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.8 MB/s   4m43spulling manifest 
pulling 6a0746a1ec1a:  77% ▕█████████████     ▏ 3.6 GB/4.7 GB  3.8 MB/s   4m

In [ ]:
# ===============================================================
# ⚙️ EXECUTION PIPELINE
# ===============================================================
os.makedirs("results", exist_ok=True)

for name, sys_prompt, user_prompt in prompts:
    print(f"\n🚀 Running consultants concurrently for {name}")
    with ThreadPoolExecutor(max_workers=len(CONSULTANT_MODELS)) as pool:
        futures = {pool.submit(run_model_for_prompt, m, sys_prompt, user_prompt, name): m for m in CONSULTANT_MODELS}
        for fut in as_completed(futures):
            _ = fut.result()
    synthesize_prompt_results(name)



🚀 Running consultants concurrently for PROMPT_1_FULL_7S
🧩 llama3     | Running PROMPT_1_FULL_7S
🧩 mistral    | Running PROMPT_1_FULL_7S
🧩 gemma2:2b  | Running PROMPT_1_FULL_7S
✅ llama3     done in 84.1s → results/PROMPT_1_FULL_7S_llama3.json
✅ mistral    done in 92.6s → results/PROMPT_1_FULL_7S_mistral.json
✅ gemma2:2b  done in 97.6s → results/PROMPT_1_FULL_7S_gemma2_2b.json

🧠 Manager (mistral) synthesizing PROMPT_1_FULL_7S...
✅ Final synthesis complete → results/PROMPT_1_FULL_7S_final.json

🚀 Running consultants concurrently for PROMPT_2_STRATEGY
🧩 mistral    | Running PROMPT_2_STRATEGY
🧩 gemma2:2b  | Running PROMPT_2_STRATEGY
🧩 llama3     | Running PROMPT_2_STRATEGY
✅ gemma2:2b  done in 68.4s → results/PROMPT_2_STRATEGY_gemma2_2b.json
✅ mistral    done in 70.6s → results/PROMPT_2_STRATEGY_mistral.json
✅ llama3     done in 71.4s → results/PROMPT_2_STRATEGY_llama3.json

🧠 Manager (mistral) synthesizing PROMPT_2_STRATEGY...
✅ Final synthesis complete → results/PROMPT_2_STRATEGY_final.

In [ ]:
# ===============================================================
# 📦 COMBINE FINAL OUTPUTS
# ===============================================================
master = []
for f in sorted(os.listdir("results")):
    if f.endswith("_final.json"):
        master.append(json.load(open(os.path.join("results", f))))
with open("results/_master_combined.json","w") as f:
    json.dump(master, f, indent=2)

print(f"\n✅ WRC 7S Analysis complete → results/_master_combined.json")


In [ ]:
# ===============================================================
# 🏁 FINAL EXECUTIVE SUMMARY
# ===============================================================
data = json.load(open("results/_master_combined.json"))
sys = "You are a senior WRC partner preparing the executive summary of a 7S diagnostic."
user = f"Here are all consolidated sections:\n{json.dumps(data, indent=2)}\n\nCreate a 1-page executive summary with key findings and recommendations."
final_report = ask_llm(MANAGER_MODEL, sys, user)

with open("results/_Executive_Summary.txt", "w") as f:
    f.write(final_report)
print("✅ Executive Summary saved → results/_Executive_Summary.txt")


In [ ]:
# ===============================================================
# 🏁 FINAL REPORT + EXECUTIVE SUMMARY (DISPLAY IN OUTPUT)
# ===============================================================
from IPython.display import Markdown, display

# 1️⃣ Load consolidated results
report_path = "results/_master_combined.json"
if not os.path.exists(report_path):
    raise SystemExit("⚠️ Combined report not found. Run the previous step first.")

data = json.load(open(report_path))

# 2️⃣ Generate executive summary
sys = (
    "You are a senior WRC (World Renowned Consultancy) partner preparing a final report. "
    "Read the synthesized results from each consultant prompt and produce:\n"
    "1. A cohesive Executive Summary (1 page max)\n"
    "2. Key findings for each 7S element\n"
    "3. Strategic recommendations.\n\n"
    "Write in a concise, structured, executive-friendly tone."
)

user = f"Here is the full synthesized 7S data:\n{json.dumps(data, indent=2, ensure_ascii=False)}"

print("🧠 Generating final executive summary...\n")
final_report = ask_llm(MANAGER_MODEL, sys, user)

# 3️⃣ Save final files
with open("results/_Executive_Summary.txt", "w") as f:
    f.write(final_report)
print("✅ Executive Summary saved → results/_Executive_Summary.txt\n")

# 4️⃣ Nicely formatted display
print("📋 ===================== WRC 7S FINAL REPORT =====================\n")
for i, section in enumerate(data, start=1):
    name = section.get("prompt")
    model = section.get("model")
    resp = section.get("response", "")[:2000]  # limit preview length
    display(Markdown(f"### 🧩 {i}. {name}  _(by {model})_\n\n{resp}\n"))

print("\n📋 ===================== EXECUTIVE SUMMARY =====================\n")
display(Markdown(final_report))

# 5️⃣ Optionally export combined markdown
combined_md = "# WRC 7S Organizational Diagnostic\n\n"
for i, sec in enumerate(data, start=1):
    combined_md += f"## {i}. {sec['prompt']}\n\n{sec['response']}\n\n"
combined_md += f"\n\n# Executive Summary\n\n{final_report}"

with open("results/_Final_Report.md", "w") as f:
    f.write(combined_md)
print("✅ Full markdown report saved → results/_Final_Report.md")
